In [1]:
#仮説検証
#正直､よくわからない､､､

import pandas as pd
import random
from src.utils import write_jsonl,random_toi,random_text_select



In [2]:
df=pd.read_csv("data/lime.cgi?down=http:%2F%2Fnlp.ist.i.kyoto-u.ac.jp%2Fnl-resource%2Frte%2Fentail_evaluation_set.txt",
               sep=" ",
              )
df.columns=["id","genre","label","s1","s2"]
df[:3]

,id,genre,label,s1,s2
0,1,語彙(体言):下位→上位,×,ジョンは怠慢で、ビルは勤勉だ。,犬は怠慢で、人は勤勉だ。
1,2,語彙(体言):下位→上位,×,羽田に行った。,千葉に行った。
2,3,語彙(体言):下位→上位,×,英文法を勉強した。,中国語を勉強した。


In [3]:
inst_list=[
"前提と仮説の関係をyes、no、unknown、undefの中から回答すること。それ以外には何も含めないことを厳守\n制約：\n- 前提が仮説を含意する場合はyes\n- 前提が仮説の否定する場合はno\n- 前提が仮説を含意せず、その否定も含意しない場合はunknown\n- 与えられた情報のみからは判断ができない場合はundef",
"前提と仮説の関係をyes、no、unknown、undefの中から回答し､それ以外には何も含めないこと｡\n制約：\n- 前提が仮説を含意する場合はyes\n- 前提が仮説の否定する場合はno\n- 前提が仮説を含意せず、その否定も含意しない場合はunknown\n- 与えられた情報のみからは判断ができない場合はundef",
"前提と仮説の関係をyes、no、unknown、undefの中から回答しなさい｡それ以外には何も含めないことを厳守しなさい｡\n制約：\n- 前提が仮説を含意する場合はyesと出力｡\n- 前提が仮説の否定する場合はnoと出力｡\n- 前提が仮説を含意せず、その否定も含意しない場合はunknownと出力｡\n- 与えられた情報のみからは判断ができない場合はundefと出力｡",
]

In [4]:
#否定
hitei_df=df[df["label"]=="×"]

target_genres=[
"語彙(体言):対義語",
"語彙(用言):副詞",
"推論:時間軸・数量",
"包含:補文",
]
hitei_df_integ= pd.DataFrame()
for genre in target_genres:
    temp_df=hitei_df[hitei_df["genre"]==genre]
    hitei_df_integ= pd.concat([hitei_df_integ, temp_df], ignore_index=True)

hitei_df_integ

,id,genre,label,s1,s2
0,128,語彙(体言):対義語,×,お腹は時計回りにさするといい。,お腹は反時計回りにさするといい。
1,129,語彙(体言):対義語,×,この道は私道だ。,この道は国道だ。
2,130,語彙(体言):対義語,×,インドゾウとアフリカゾウのどこが違うか教えて下さい。,インドゾウとアフリカゾウはそっくりだ。
3,131,語彙(体言):対義語,×,カリフォルニアはアメリカの西海岸にある。,カリフォルニアはアメリカの東海岸にある。
4,132,語彙(体言):対義語,×,現金は流動資産だ。,現金は固定資産だ。
...,...,...,...,...,...
58,2376,包含:補文,×,彼は私が転んだと勘違いした。,私は転んだ。
59,2377,包含:補文,×,彼は本当は嫌いだったと告げた。,本当は遊びに行きたかった。
60,2378,包含:補文,×,母は浦島太郎が亀を助けたことを話した。,母は亀を助けた。
61,2379,包含:補文,×,両親は私が法学部へ進んだことを喜んだ。,私は工学部へ進んだ。


In [5]:

problem_list=[]
for d in iter(hitei_df_integ.values):
    s1=d[3]
    s2=d[4]
        
    #作文
    q_text=f"前提：{s1}\n仮説：{s2}"

    dd={
        "instruction":random.choice(inst_list),
        "question":q_text,
        "answer":"no",
    }
    problem_list.append(dd)

write_jsonl(problem_list,"data/jsonl/kasetsu_no.jsonl")

In [6]:

#unk
sel_df=df[df["label"]=="×"]

target_genres=[
"語彙(体言):下位→上位",
"語彙(体言):上位→下位",
"語彙(体言):性質",
"語彙(体言):同義語",
"語彙(体言):名詞の格関係",
"語彙(用言):前提的",
"語彙(用言):副詞",
"推論:副助詞+対応",
"包含:節",
"包含:名詞句",

]
integ_df= pd.DataFrame()
for genre in target_genres:
    temp_df=sel_df[sel_df["genre"]==genre]
    integ_df= pd.concat([integ_df, temp_df], ignore_index=True)


problem_list=[]
for d in iter(integ_df.values):
    s1=d[3]
    s2=d[4]
        
    #作文
    q_text=f"前提：{s1}\n仮説：{s2}"

    dd={
        "instruction":random.choice(inst_list),
        "question":q_text,
        "answer":"unknown",
    }
    problem_list.append(dd)

write_jsonl(problem_list,"data/jsonl/kasetsu_unk.jsonl")


#undef
sel_df=df[df["label"]=="×"]

target_genres=[
"語彙(用言):含意",
"語彙(用言):言い換え",
"構文:主語の変換",
"構文:複文の変換",
"推論:結果→原因",
"推論:原因→結果",
"推論:順接・逆接+一般化",
"推論:順接・逆接+具体化",
]
integ_df= pd.DataFrame()
for genre in target_genres:
    temp_df=sel_df[sel_df["genre"]==genre]
    integ_df= pd.concat([integ_df, temp_df], ignore_index=True)

problem_list=[]
for d in iter(integ_df.values):
    s1=d[3]
    s2=d[4]
        
    #作文
    q_text=f"前提：{s1}\n仮説：{s2}"

    dd={
        "instruction":random.choice(inst_list),
        "question":q_text,
        "answer":"undef",
    }
    problem_list.append(dd)

write_jsonl(problem_list,"data/jsonl/kasetsu_undef.jsonl")

In [7]:
#good
sel_df=df[df["label"]=="◎"]

problem_list=[]
for d in iter(sel_df.values):
    s1=d[3]
    s2=d[4]
        
    #作文
    q_text=f"前提：{s1}\n仮説：{s2}"

    dd={
        "instruction":random.choice(inst_list),
        "question":q_text,
        "answer":"yes",
    }
    problem_list.append(dd)

write_jsonl(problem_list,"data/jsonl/kasetsu_yes.jsonl")

In [8]:
integ_df

,id,genre,label,s1,s2
0,502,語彙(用言):含意,×,このポスターはデザインがいい。,センスがない。
1,503,語彙(用言):含意,×,旺盛な食欲はとどまるところを知らなかった。,食欲は知ることができる。
2,504,語彙(用言):含意,×,化粧は女を化かす。,化粧は恐ろしい。
3,505,語彙(用言):含意,×,街路樹のイチョウが色づいた。,街路樹のイチョウには色が無かった。
4,506,語彙(用言):含意,×,紅葉が色づいた。,紅葉には色が無かった。
...,...,...,...,...,...
181,1908,推論:順接・逆接+具体化,×,女性ではないのなら、男性である。,まなみさんは女性である。
182,1909,推論:順接・逆接+具体化,×,女性ではないのなら、男性である。,奈月さんは男性である。
183,1910,推論:順接・逆接+具体化,×,太郎は甘いものが好きだが、プラリネは嫌いだ。,プラリネはおいしい。
184,1911,推論:順接・逆接+具体化,×,太郎は甘いものが好きだが、プラリネは嫌いだ。,プラリネは甘くない。
